# Day 7: Portfolio Optimization Interview Questions & Week Summary

## Week 18 - Portfolio Optimization

**Date:** Day 7 of Week 18  
**Focus:** Interview preparation with 10 essential portfolio optimization questions

### Topics Covered:
1. Mean-Variance Optimization (Markowitz)
2. Portfolio Expected Return & Variance
3. Efficient Frontier Construction
4. Sharpe Ratio Optimization
5. Minimum Variance Portfolio
6. Maximum Sharpe Ratio (Tangency) Portfolio
7. Constrained Portfolio Optimization
8. Risk Parity Portfolio
9. Black-Litterman Model
10. Week Summary & Key Formulas

---

## Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize, minimize_scalar
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

In [ ]:
# Create sample data for portfolio optimization examples
def generate_sample_data(n_assets=5, n_periods=252*3):
    """Generate sample asset returns data for portfolio optimization."""
    
    # Asset names
    assets = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']
    
    # True expected returns (annualized)
    true_returns = np.array([0.12, 0.10, 0.08, 0.07, 0.09])
    
    # True volatilities (annualized)
    true_vols = np.array([0.25, 0.20, 0.18, 0.22, 0.15])
    
    # Correlation matrix
    corr_matrix = np.array([
        [1.00, 0.50, 0.40, 0.30, 0.45],
        [0.50, 1.00, 0.35, 0.25, 0.40],
        [0.40, 0.35, 1.00, 0.45, 0.50],
        [0.30, 0.25, 0.45, 1.00, 0.35],
        [0.45, 0.40, 0.50, 0.35, 1.00]
    ])
    
    # Covariance matrix
    cov_matrix = np.outer(true_vols, true_vols) * corr_matrix
    
    # Generate daily returns
    daily_returns = true_returns / 252
    daily_cov = cov_matrix / 252
    
    returns_data = np.random.multivariate_normal(daily_returns, daily_cov, n_periods)
    
    # Create DataFrame
    dates = pd.date_range(start='2022-01-01', periods=n_periods, freq='B')
    returns_df = pd.DataFrame(returns_data, index=dates, columns=assets)
    
    return returns_df, assets

# Generate sample data
returns_df, asset_names = generate_sample_data()

# Calculate statistics from sample data
expected_returns = returns_df.mean() * 252  # Annualized
cov_matrix = returns_df.cov() * 252  # Annualized
std_devs = returns_df.std() * np.sqrt(252)  # Annualized
corr_matrix = returns_df.corr()

print("Sample Portfolio Data Generated")
print("=" * 50)
print(f"\nAssets: {asset_names}")
print(f"\nExpected Annual Returns:\n{expected_returns}")
print(f"\nAnnual Standard Deviations:\n{std_devs}")
print(f"\nCorrelation Matrix:\n{corr_matrix.round(3)}")

---

## Question 1: Explain Mean-Variance Optimization

**Interview Question:** *"Explain Markowitz Mean-Variance Optimization and its core assumptions. How would you implement a basic portfolio weight calculation?"*

### Key Concepts:

**Mean-Variance Optimization (MVO)** is Harry Markowitz's framework (1952) for constructing portfolios that maximize expected return for a given level of risk.

**Core Assumptions:**
1. Investors are risk-averse and prefer higher returns
2. Returns are normally distributed
3. Investors only care about mean and variance of returns
4. Markets are frictionless (no transaction costs)
5. Single-period investment horizon

**The Optimization Problem:**

$$\min_{w} \quad w^T \Sigma w$$
$$\text{subject to:} \quad w^T \mu = \mu_{\text{target}}$$
$$\quad \quad \quad \quad \mathbf{1}^T w = 1$$

Where:
- $w$ = portfolio weights
- $\Sigma$ = covariance matrix
- $\mu$ = expected returns vector
- $\mu_{\text{target}}$ = target portfolio return

In [ ]:
# Question 1: Mean-Variance Optimization Implementation

def mean_variance_optimization(expected_returns, cov_matrix, target_return):
    """
    Solve the mean-variance optimization problem for a given target return.
    
    Parameters:
    -----------
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix of returns
    target_return : float
        Target portfolio return
        
    Returns:
    --------
    dict : Contains optimal weights, return, volatility, and Sharpe ratio
    """
    n_assets = len(expected_returns)
    
    # Objective function: minimize portfolio variance
    def portfolio_variance(weights):
        return weights @ cov_matrix @ weights
    
    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Weights sum to 1
        {'type': 'eq', 'fun': lambda w: w @ expected_returns - target_return}  # Target return
    ]
    
    # Bounds (allow short selling for now)
    bounds = tuple((-1, 1) for _ in range(n_assets))
    
    # Initial guess: equal weights
    initial_weights = np.ones(n_assets) / n_assets
    
    # Optimize
    result = minimize(
        portfolio_variance,
        initial_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    optimal_weights = result.x
    portfolio_return = optimal_weights @ expected_returns
    portfolio_vol = np.sqrt(optimal_weights @ cov_matrix @ optimal_weights)
    sharpe_ratio = portfolio_return / portfolio_vol  # Assuming rf = 0
    
    return {
        'weights': optimal_weights,
        'return': portfolio_return,
        'volatility': portfolio_vol,
        'sharpe_ratio': sharpe_ratio,
        'success': result.success
    }

# Example: Find optimal portfolio for 10% target return
target = 0.10
result = mean_variance_optimization(expected_returns.values, cov_matrix.values, target)

print("Question 1: Mean-Variance Optimization")
print("=" * 50)
print(f"\nTarget Return: {target:.2%}")
print(f"\nOptimal Portfolio Weights:")
for asset, weight in zip(asset_names, result['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nPortfolio Statistics:")
print(f"  Expected Return: {result['return']:.4f} ({result['return']:.2%})")
print(f"  Volatility: {result['volatility']:.4f} ({result['volatility']:.2%})")
print(f"  Sharpe Ratio: {result['sharpe_ratio']:.4f}")
print(f"  Optimization Success: {result['success']}")

---

## Question 2: Calculate Portfolio Expected Return

**Interview Question:** *"Given a set of asset weights and expected returns, how do you calculate the portfolio's expected return? Derive the formula and implement it."*

### Formula:

The portfolio expected return is the weighted average of individual asset returns:

$$E[R_p] = \sum_{i=1}^{n} w_i \cdot E[R_i] = w^T \mu$$

Where:
- $w_i$ = weight of asset $i$
- $E[R_i]$ = expected return of asset $i$
- $n$ = number of assets

In [ ]:
# Question 2: Portfolio Expected Return Calculation

def calculate_portfolio_return(weights, expected_returns):
    """
    Calculate portfolio expected return.
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights for each asset
    expected_returns : array-like
        Expected returns for each asset
        
    Returns:
    --------
    float : Portfolio expected return
    """
    weights = np.array(weights)
    expected_returns = np.array(expected_returns)
    return np.dot(weights, expected_returns)

# Example calculations
print("Question 2: Portfolio Expected Return")
print("=" * 50)

# Example 1: Equal-weighted portfolio
equal_weights = np.ones(len(asset_names)) / len(asset_names)
equal_return = calculate_portfolio_return(equal_weights, expected_returns.values)

print(f"\nExample 1: Equal-Weighted Portfolio")
print(f"Weights: {dict(zip(asset_names, equal_weights))}")
print(f"Portfolio Expected Return: {equal_return:.4f} ({equal_return:.2%})")

# Example 2: Custom weights (60/40 style with more assets)
custom_weights = np.array([0.30, 0.25, 0.20, 0.15, 0.10])
custom_return = calculate_portfolio_return(custom_weights, expected_returns.values)

print(f"\nExample 2: Custom-Weighted Portfolio")
print(f"Weights: {dict(zip(asset_names, custom_weights))}")
print(f"Portfolio Expected Return: {custom_return:.4f} ({custom_return:.2%})")

# Example 3: Concentrated portfolio
concentrated_weights = np.array([0.50, 0.30, 0.10, 0.05, 0.05])
concentrated_return = calculate_portfolio_return(concentrated_weights, expected_returns.values)

print(f"\nExample 3: Concentrated Portfolio (50% Tech)")
print(f"Weights: {dict(zip(asset_names, concentrated_weights))}")
print(f"Portfolio Expected Return: {concentrated_return:.4f} ({concentrated_return:.2%})")

# Verify using matrix multiplication
print(f"\nVerification using matrix notation (w^T * μ):")
print(f"Equal-weighted: {equal_weights @ expected_returns.values:.4f}")
print(f"Custom: {custom_weights @ expected_returns.values:.4f}")
print(f"Concentrated: {concentrated_weights @ expected_returns.values:.4f}")

---

## Question 3: Compute Portfolio Variance and Covariance Matrix

**Interview Question:** *"How do you calculate portfolio variance? Explain the role of the covariance matrix and demonstrate how to estimate it from historical returns."*

### Formula:

Portfolio variance captures how portfolio return varies, accounting for correlations between assets:

$$\sigma_p^2 = w^T \Sigma w = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij}$$

Where:
- $\sigma_p^2$ = portfolio variance
- $\Sigma$ = covariance matrix
- $\sigma_{ij}$ = covariance between assets $i$ and $j$

**Covariance Matrix Estimation:**
$$\hat{\Sigma} = \frac{1}{T-1} \sum_{t=1}^{T} (r_t - \bar{r})(r_t - \bar{r})^T$$

In [ ]:
# Question 3: Portfolio Variance and Covariance Matrix

def calculate_portfolio_variance(weights, cov_matrix):
    """
    Calculate portfolio variance using matrix notation.
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights
    cov_matrix : array-like
        Covariance matrix
        
    Returns:
    --------
    float : Portfolio variance
    """
    weights = np.array(weights)
    cov_matrix = np.array(cov_matrix)
    return weights @ cov_matrix @ weights

def calculate_portfolio_volatility(weights, cov_matrix):
    """Calculate portfolio volatility (standard deviation)."""
    return np.sqrt(calculate_portfolio_variance(weights, cov_matrix))

def estimate_covariance_matrix(returns_df, annualize=True):
    """
    Estimate covariance matrix from historical returns.
    
    Parameters:
    -----------
    returns_df : DataFrame
        Historical returns data
    annualize : bool
        Whether to annualize the covariance matrix
        
    Returns:
    --------
    DataFrame : Estimated covariance matrix
    """
    cov = returns_df.cov()
    if annualize:
        cov = cov * 252  # Assuming daily data
    return cov

print("Question 3: Portfolio Variance and Covariance Matrix")
print("=" * 50)

# Display the covariance matrix
print("\nEstimated Covariance Matrix (Annualized):")
print(cov_matrix.round(4))

# Calculate variance for different portfolios
portfolios = {
    'Equal Weight': np.ones(5) / 5,
    'Tech Heavy': np.array([0.50, 0.20, 0.15, 0.10, 0.05]),
    'Defensive': np.array([0.10, 0.30, 0.20, 0.10, 0.30])
}

print("\nPortfolio Variance & Volatility Comparison:")
print("-" * 50)
for name, weights in portfolios.items():
    variance = calculate_portfolio_variance(weights, cov_matrix.values)
    volatility = calculate_portfolio_volatility(weights, cov_matrix.values)
    print(f"{name}:")
    print(f"  Variance: {variance:.6f}")
    print(f"  Volatility: {volatility:.4f} ({volatility:.2%})")
    print()

# Show manual variance calculation (expanded form)
print("\nManual Variance Calculation (Equal Weight):")
w = equal_weights
cov = cov_matrix.values
manual_var = 0
for i in range(len(w)):
    for j in range(len(w)):
        contribution = w[i] * w[j] * cov[i, j]
        manual_var += contribution
print(f"Sum of w_i * w_j * σ_ij = {manual_var:.6f}")
print(f"Matrix formula w'Σw = {w @ cov @ w:.6f}")

---

## Question 4: Implement Efficient Frontier

**Interview Question:** *"What is the efficient frontier and how would you generate it programmatically?"*

### Key Concepts:

The **Efficient Frontier** represents the set of portfolios that offer:
- Maximum expected return for a given level of risk, OR
- Minimum risk for a given level of expected return

**Algorithm to Generate Efficient Frontier:**
1. Determine the range of achievable returns (from min to max)
2. For each target return, solve for minimum variance portfolio
3. Plot the risk-return pairs

All portfolios below the efficient frontier are **dominated** (suboptimal).

In [ ]:
# Question 4: Efficient Frontier Implementation

def generate_efficient_frontier(expected_returns, cov_matrix, n_points=100, 
                                 allow_short=True, rf_rate=0.02):
    """
    Generate the efficient frontier.
    
    Parameters:
    -----------
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix
    n_points : int
        Number of points on the frontier
    allow_short : bool
        Whether to allow short selling
    rf_rate : float
        Risk-free rate for Sharpe ratio calculation
        
    Returns:
    --------
    dict : Contains returns, volatilities, weights, and Sharpe ratios
    """
    n_assets = len(expected_returns)
    expected_returns = np.array(expected_returns)
    cov_matrix = np.array(cov_matrix)
    
    # Define return range
    min_ret = expected_returns.min()
    max_ret = expected_returns.max()
    target_returns = np.linspace(min_ret * 0.8, max_ret * 1.2, n_points)
    
    frontier_returns = []
    frontier_volatilities = []
    frontier_weights = []
    frontier_sharpe = []
    
    for target_ret in target_returns:
        # Objective: minimize variance
        def portfolio_variance(w):
            return w @ cov_matrix @ w
        
        # Constraints
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
            {'type': 'eq', 'fun': lambda w, t=target_ret: w @ expected_returns - t}
        ]
        
        # Bounds
        if allow_short:
            bounds = tuple((-1, 1) for _ in range(n_assets))
        else:
            bounds = tuple((0, 1) for _ in range(n_assets))
        
        # Initial guess
        init_weights = np.ones(n_assets) / n_assets
        
        # Optimize
        result = minimize(
            portfolio_variance,
            init_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000}
        )
        
        if result.success:
            w = result.x
            ret = w @ expected_returns
            vol = np.sqrt(w @ cov_matrix @ w)
            sharpe = (ret - rf_rate) / vol
            
            frontier_returns.append(ret)
            frontier_volatilities.append(vol)
            frontier_weights.append(w)
            frontier_sharpe.append(sharpe)
    
    return {
        'returns': np.array(frontier_returns),
        'volatilities': np.array(frontier_volatilities),
        'weights': np.array(frontier_weights),
        'sharpe_ratios': np.array(frontier_sharpe)
    }

# Generate efficient frontier
ef = generate_efficient_frontier(expected_returns.values, cov_matrix.values, n_points=50)

# Plot the efficient frontier
fig, ax = plt.subplots(figsize=(12, 8))

# Plot efficient frontier
scatter = ax.scatter(ef['volatilities'], ef['returns'], 
                     c=ef['sharpe_ratios'], cmap='viridis', s=50, alpha=0.8)
ax.plot(ef['volatilities'], ef['returns'], 'b-', linewidth=2, label='Efficient Frontier')

# Plot individual assets
for i, asset in enumerate(asset_names):
    ax.scatter(std_devs.values[i], expected_returns.values[i], 
               marker='*', s=300, label=asset, zorder=5)

# Find and mark maximum Sharpe ratio portfolio
max_sharpe_idx = np.argmax(ef['sharpe_ratios'])
ax.scatter(ef['volatilities'][max_sharpe_idx], ef['returns'][max_sharpe_idx],
           marker='D', s=200, c='red', edgecolors='black', linewidth=2,
           label='Max Sharpe Ratio', zorder=6)

# Find and mark minimum variance portfolio
min_vol_idx = np.argmin(ef['volatilities'])
ax.scatter(ef['volatilities'][min_vol_idx], ef['returns'][min_vol_idx],
           marker='s', s=200, c='green', edgecolors='black', linewidth=2,
           label='Min Variance', zorder=6)

plt.colorbar(scatter, label='Sharpe Ratio')
ax.set_xlabel('Volatility (Standard Deviation)', fontsize=12)
ax.set_ylabel('Expected Return', fontsize=12)
ax.set_title('Question 4: Efficient Frontier', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEfficient Frontier Statistics:")
print(f"Number of efficient portfolios: {len(ef['returns'])}")
print(f"Return range: {ef['returns'].min():.2%} to {ef['returns'].max():.2%}")
print(f"Volatility range: {ef['volatilities'].min():.2%} to {ef['volatilities'].max():.2%}")

---

## Question 5: Calculate Sharpe Ratio

**Interview Question:** *"What is the Sharpe Ratio? How do you calculate it and what does it tell you about portfolio performance?"*

### Formula:

The **Sharpe Ratio** measures risk-adjusted return:

$$SR = \frac{E[R_p] - R_f}{\sigma_p}$$

Where:
- $E[R_p]$ = expected portfolio return
- $R_f$ = risk-free rate
- $\sigma_p$ = portfolio standard deviation

**Interpretation:**
- Higher Sharpe ratio = better risk-adjusted performance
- SR > 1 is generally considered good
- SR > 2 is excellent
- Allows comparison across different strategies/assets

In [ ]:
# Question 5: Sharpe Ratio Calculation

def calculate_sharpe_ratio(returns, volatility, rf_rate=0.02):
    """
    Calculate the Sharpe Ratio.
    
    Parameters:
    -----------
    returns : float or array-like
        Portfolio or asset expected return (annualized)
    volatility : float or array-like
        Portfolio or asset volatility (annualized)
    rf_rate : float
        Risk-free rate (annualized)
        
    Returns:
    --------
    float or array : Sharpe ratio(s)
    """
    return (returns - rf_rate) / volatility

def calculate_sharpe_from_returns(returns_series, rf_rate=0.02, periods_per_year=252):
    """
    Calculate Sharpe Ratio from a returns time series.
    
    Parameters:
    -----------
    returns_series : array-like
        Time series of returns
    rf_rate : float
        Annualized risk-free rate
    periods_per_year : int
        Number of periods per year (252 for daily, 12 for monthly)
        
    Returns:
    --------
    float : Annualized Sharpe ratio
    """
    excess_returns = returns_series - rf_rate / periods_per_year
    mean_excess = excess_returns.mean() * periods_per_year
    std_excess = returns_series.std() * np.sqrt(periods_per_year)
    return mean_excess / std_excess

print("Question 5: Sharpe Ratio Calculation")
print("=" * 50)

rf_rate = 0.02  # 2% risk-free rate

# Calculate Sharpe ratios for individual assets
print("\nIndividual Asset Sharpe Ratios:")
print("-" * 40)
for asset in asset_names:
    ret = expected_returns[asset]
    vol = std_devs[asset]
    sharpe = calculate_sharpe_ratio(ret, vol, rf_rate)
    print(f"{asset}: Return={ret:.2%}, Vol={vol:.2%}, Sharpe={sharpe:.3f}")

# Calculate for different portfolios
print("\nPortfolio Sharpe Ratios:")
print("-" * 40)

for name, weights in portfolios.items():
    port_ret = weights @ expected_returns.values
    port_vol = np.sqrt(weights @ cov_matrix.values @ weights)
    sharpe = calculate_sharpe_ratio(port_ret, port_vol, rf_rate)
    print(f"{name}: Return={port_ret:.2%}, Vol={port_vol:.2%}, Sharpe={sharpe:.3f}")

# Calculate Sharpe from actual returns time series
print("\nSharpe Ratio from Historical Returns (Equal-Weight Portfolio):")
print("-" * 40)
portfolio_returns = returns_df @ equal_weights
hist_sharpe = calculate_sharpe_from_returns(portfolio_returns, rf_rate)
print(f"Annualized Sharpe Ratio: {hist_sharpe:.3f}")

# Sharpe ratio interpretation
print("\nSharpe Ratio Interpretation Guide:")
print("-" * 40)
print("SR < 0.5  : Poor")
print("0.5 ≤ SR < 1.0 : Acceptable")
print("1.0 ≤ SR < 2.0 : Good")
print("SR ≥ 2.0 : Excellent")

---

## Question 6: Find Minimum Variance Portfolio

**Interview Question:** *"How do you find the Global Minimum Variance Portfolio? Provide both analytical and numerical solutions."*

### Analytical Solution:

For unconstrained minimum variance (weights sum to 1):

$$w_{MVP} = \frac{\Sigma^{-1} \mathbf{1}}{\mathbf{1}^T \Sigma^{-1} \mathbf{1}}$$

Where:
- $\Sigma^{-1}$ = inverse of covariance matrix
- $\mathbf{1}$ = vector of ones

### Numerical Solution:
Use optimization to minimize $w^T \Sigma w$ subject to $\sum w_i = 1$

In [ ]:
# Question 6: Minimum Variance Portfolio

def minimum_variance_portfolio_analytical(cov_matrix):
    """
    Calculate the Global Minimum Variance Portfolio analytically.
    
    Parameters:
    -----------
    cov_matrix : array-like
        Covariance matrix
        
    Returns:
    --------
    dict : Contains weights, variance, and volatility
    """
    cov_matrix = np.array(cov_matrix)
    n = cov_matrix.shape[0]
    ones = np.ones(n)
    
    # Analytical solution: w = Σ^(-1) * 1 / (1' * Σ^(-1) * 1)
    cov_inv = np.linalg.inv(cov_matrix)
    weights = cov_inv @ ones / (ones @ cov_inv @ ones)
    
    variance = weights @ cov_matrix @ weights
    volatility = np.sqrt(variance)
    
    return {
        'weights': weights,
        'variance': variance,
        'volatility': volatility
    }

def minimum_variance_portfolio_numerical(cov_matrix, long_only=False):
    """
    Calculate the Global Minimum Variance Portfolio numerically.
    
    Parameters:
    -----------
    cov_matrix : array-like
        Covariance matrix
    long_only : bool
        Whether to constrain weights to be non-negative
        
    Returns:
    --------
    dict : Contains weights, variance, volatility, and success flag
    """
    cov_matrix = np.array(cov_matrix)
    n = cov_matrix.shape[0]
    
    # Objective: minimize variance
    def portfolio_variance(w):
        return w @ cov_matrix @ w
    
    # Constraint: weights sum to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds
    if long_only:
        bounds = tuple((0, 1) for _ in range(n))
    else:
        bounds = tuple((-1, 1) for _ in range(n))
    
    # Initial guess
    init_weights = np.ones(n) / n
    
    # Optimize
    result = minimize(
        portfolio_variance,
        init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    weights = result.x
    variance = weights @ cov_matrix @ weights
    volatility = np.sqrt(variance)
    
    return {
        'weights': weights,
        'variance': variance,
        'volatility': volatility,
        'success': result.success
    }

print("Question 6: Minimum Variance Portfolio")
print("=" * 50)

# Analytical solution (allows short selling)
analytical = minimum_variance_portfolio_analytical(cov_matrix.values)

print("\n1. Analytical Solution (Short Selling Allowed):")
print("-" * 40)
print("Weights:")
for asset, weight in zip(asset_names, analytical['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nPortfolio Variance: {analytical['variance']:.6f}")
print(f"Portfolio Volatility: {analytical['volatility']:.4f} ({analytical['volatility']:.2%})")

# Numerical solution (allows short selling)
numerical_short = minimum_variance_portfolio_numerical(cov_matrix.values, long_only=False)

print("\n2. Numerical Solution (Short Selling Allowed):")
print("-" * 40)
print("Weights:")
for asset, weight in zip(asset_names, numerical_short['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nPortfolio Volatility: {numerical_short['volatility']:.4f} ({numerical_short['volatility']:.2%})")

# Numerical solution (long only)
numerical_long = minimum_variance_portfolio_numerical(cov_matrix.values, long_only=True)

print("\n3. Numerical Solution (Long Only - No Short Selling):")
print("-" * 40)
print("Weights:")
for asset, weight in zip(asset_names, numerical_long['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nPortfolio Volatility: {numerical_long['volatility']:.4f} ({numerical_long['volatility']:.2%})")

# Verify analytical matches numerical
print("\n4. Verification:")
print("-" * 40)
diff = np.max(np.abs(analytical['weights'] - numerical_short['weights']))
print(f"Max weight difference (analytical vs numerical): {diff:.2e}")
print("Solutions match!" if diff < 1e-6 else "Solutions differ!")

---

## Question 7: Implement Maximum Sharpe Ratio Portfolio

**Interview Question:** *"How do you find the Maximum Sharpe Ratio (Tangency) Portfolio? Why is it called the tangency portfolio?"*

### Key Concepts:

The **Maximum Sharpe Ratio Portfolio** (or **Tangency Portfolio**) is the portfolio on the efficient frontier that has the highest Sharpe ratio.

**Why "Tangency"?**
- It's the point where the Capital Market Line (CML) is tangent to the efficient frontier
- Any investor can combine this portfolio with the risk-free asset to achieve their desired risk level

**Optimization Problem:**

$$\max_{w} \frac{w^T \mu - R_f}{\sqrt{w^T \Sigma w}}$$

Subject to: $\sum w_i = 1$

In [ ]:
# Question 7: Maximum Sharpe Ratio (Tangency) Portfolio

def max_sharpe_portfolio(expected_returns, cov_matrix, rf_rate=0.02, long_only=False):
    """
    Find the Maximum Sharpe Ratio (Tangency) Portfolio.
    
    Parameters:
    -----------
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix
    rf_rate : float
        Risk-free rate
    long_only : bool
        Whether to constrain weights to be non-negative
        
    Returns:
    --------
    dict : Contains weights, return, volatility, and Sharpe ratio
    """
    expected_returns = np.array(expected_returns)
    cov_matrix = np.array(cov_matrix)
    n = len(expected_returns)
    
    # Objective: minimize negative Sharpe ratio (maximize Sharpe)
    def neg_sharpe_ratio(w):
        port_ret = w @ expected_returns
        port_vol = np.sqrt(w @ cov_matrix @ w)
        return -(port_ret - rf_rate) / port_vol
    
    # Constraint: weights sum to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds
    if long_only:
        bounds = tuple((0, 1) for _ in range(n))
    else:
        bounds = tuple((-1, 1) for _ in range(n))
    
    # Initial guess
    init_weights = np.ones(n) / n
    
    # Optimize
    result = minimize(
        neg_sharpe_ratio,
        init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000}
    )
    
    weights = result.x
    port_return = weights @ expected_returns
    port_vol = np.sqrt(weights @ cov_matrix @ weights)
    sharpe = (port_return - rf_rate) / port_vol
    
    return {
        'weights': weights,
        'return': port_return,
        'volatility': port_vol,
        'sharpe_ratio': sharpe,
        'success': result.success
    }

print("Question 7: Maximum Sharpe Ratio Portfolio")
print("=" * 50)

rf_rate = 0.02

# Find max Sharpe portfolio (allow short selling)
max_sharpe_short = max_sharpe_portfolio(
    expected_returns.values, cov_matrix.values, rf_rate, long_only=False
)

print("\n1. Maximum Sharpe Portfolio (Short Selling Allowed):")
print("-" * 40)
print("Weights:")
for asset, weight in zip(asset_names, max_sharpe_short['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nExpected Return: {max_sharpe_short['return']:.4f} ({max_sharpe_short['return']:.2%})")
print(f"Volatility: {max_sharpe_short['volatility']:.4f} ({max_sharpe_short['volatility']:.2%})")
print(f"Sharpe Ratio: {max_sharpe_short['sharpe_ratio']:.4f}")

# Find max Sharpe portfolio (long only)
max_sharpe_long = max_sharpe_portfolio(
    expected_returns.values, cov_matrix.values, rf_rate, long_only=True
)

print("\n2. Maximum Sharpe Portfolio (Long Only):")
print("-" * 40)
print("Weights:")
for asset, weight in zip(asset_names, max_sharpe_long['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"\nExpected Return: {max_sharpe_long['return']:.4f} ({max_sharpe_long['return']:.2%})")
print(f"Volatility: {max_sharpe_long['volatility']:.4f} ({max_sharpe_long['volatility']:.2%})")
print(f"Sharpe Ratio: {max_sharpe_long['sharpe_ratio']:.4f}")

# Visualize Capital Market Line
fig, ax = plt.subplots(figsize=(12, 8))

# Plot efficient frontier
ax.scatter(ef['volatilities'], ef['returns'], c=ef['sharpe_ratios'], 
           cmap='viridis', s=30, alpha=0.7, label='Efficient Frontier')

# Plot individual assets
for i, asset in enumerate(asset_names):
    ax.scatter(std_devs.values[i], expected_returns.values[i], 
               marker='*', s=200, zorder=5)
    ax.annotate(asset, (std_devs.values[i], expected_returns.values[i]), 
                xytext=(5, 5), textcoords='offset points')

# Plot tangency portfolio
ax.scatter(max_sharpe_long['volatility'], max_sharpe_long['return'],
           marker='D', s=200, c='red', edgecolors='black', linewidth=2,
           label=f"Tangency Portfolio (SR={max_sharpe_long['sharpe_ratio']:.2f})", zorder=6)

# Plot Capital Market Line
cml_x = np.linspace(0, max(ef['volatilities']) * 1.2, 100)
cml_y = rf_rate + max_sharpe_long['sharpe_ratio'] * cml_x
ax.plot(cml_x, cml_y, 'r--', linewidth=2, label='Capital Market Line')

# Plot risk-free rate
ax.scatter(0, rf_rate, marker='o', s=150, c='green', edgecolors='black',
           linewidth=2, label=f'Risk-Free Rate ({rf_rate:.1%})', zorder=6)

ax.set_xlabel('Volatility', fontsize=12)
ax.set_ylabel('Expected Return', fontsize=12)
ax.set_title('Question 7: Maximum Sharpe Ratio & Capital Market Line', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

---

## Question 8: Apply Portfolio Constraints

**Interview Question:** *"How do you handle real-world constraints in portfolio optimization? Implement a constrained optimization with position limits and sector exposure constraints."*

### Common Constraints:

1. **Long-Only:** $w_i \geq 0$ for all assets
2. **Position Limits:** $L_i \leq w_i \leq U_i$ (min/max weight per asset)
3. **Sector Exposure:** $\sum_{i \in S} w_i \leq c_S$ (sector cap)
4. **Turnover Constraint:** $\sum |w_i - w_i^{old}| \leq T$
5. **Cardinality:** Maximum number of assets (NP-hard)

In [ ]:
# Question 8: Constrained Portfolio Optimization

def constrained_portfolio_optimization(expected_returns, cov_matrix, rf_rate=0.02,
                                        min_weight=0.0, max_weight=0.30,
                                        sector_mapping=None, sector_limits=None,
                                        target_return=None):
    """
    Optimize portfolio with multiple constraints.
    
    Parameters:
    -----------
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix
    rf_rate : float
        Risk-free rate
    min_weight : float
        Minimum weight per asset
    max_weight : float
        Maximum weight per asset
    sector_mapping : dict
        Maps each asset index to a sector
    sector_limits : dict
        Maximum allocation per sector
    target_return : float
        Target portfolio return (if None, maximize Sharpe)
        
    Returns:
    --------
    dict : Optimization results
    """
    expected_returns = np.array(expected_returns)
    cov_matrix = np.array(cov_matrix)
    n = len(expected_returns)
    
    # Objective function
    if target_return is None:
        # Maximize Sharpe ratio
        def objective(w):
            port_ret = w @ expected_returns
            port_vol = np.sqrt(w @ cov_matrix @ w)
            return -(port_ret - rf_rate) / port_vol
    else:
        # Minimize variance for target return
        def objective(w):
            return w @ cov_matrix @ w
    
    # Constraints list
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights sum to 1
    ]
    
    # Add target return constraint if specified
    if target_return is not None:
        constraints.append({
            'type': 'eq', 
            'fun': lambda w, t=target_return: w @ expected_returns - t
        })
    
    # Add sector constraints
    if sector_mapping is not None and sector_limits is not None:
        for sector, limit in sector_limits.items():
            sector_indices = [i for i, s in sector_mapping.items() if s == sector]
            if sector_indices:
                constraints.append({
                    'type': 'ineq',
                    'fun': lambda w, idx=sector_indices, lim=limit: lim - sum(w[i] for i in idx)
                })
    
    # Bounds (position limits)
    bounds = tuple((min_weight, max_weight) for _ in range(n))
    
    # Initial guess
    init_weights = np.ones(n) / n
    
    # Optimize
    result = minimize(
        objective,
        init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000}
    )
    
    weights = result.x
    port_return = weights @ expected_returns
    port_vol = np.sqrt(weights @ cov_matrix @ weights)
    sharpe = (port_return - rf_rate) / port_vol
    
    return {
        'weights': weights,
        'return': port_return,
        'volatility': port_vol,
        'sharpe_ratio': sharpe,
        'success': result.success,
        'message': result.message
    }

print("Question 8: Constrained Portfolio Optimization")
print("=" * 50)

# Define sector mapping
sector_mapping = {
    0: 'Growth',     # Tech
    1: 'Defensive',  # Healthcare
    2: 'Value',      # Finance
    3: 'Cyclical',   # Energy
    4: 'Defensive'   # Consumer
}

sector_limits = {
    'Growth': 0.35,
    'Defensive': 0.50,
    'Value': 0.30,
    'Cyclical': 0.20
}

# Case 1: Unconstrained (long only)
print("\n1. Unconstrained (Long Only, Max Sharpe):")
print("-" * 40)
unconstrained = constrained_portfolio_optimization(
    expected_returns.values, cov_matrix.values,
    min_weight=0.0, max_weight=1.0
)
for asset, weight in zip(asset_names, unconstrained['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"Sharpe Ratio: {unconstrained['sharpe_ratio']:.4f}")

# Case 2: Position limits (min 5%, max 30%)
print("\n2. Position Limits (5% ≤ w ≤ 30%):")
print("-" * 40)
pos_limited = constrained_portfolio_optimization(
    expected_returns.values, cov_matrix.values,
    min_weight=0.05, max_weight=0.30
)
for asset, weight in zip(asset_names, pos_limited['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")
print(f"Sharpe Ratio: {pos_limited['sharpe_ratio']:.4f}")

# Case 3: With sector constraints
print("\n3. Position Limits + Sector Constraints:")
print(f"   Sector Limits: {sector_limits}")
print("-" * 40)
sector_constrained = constrained_portfolio_optimization(
    expected_returns.values, cov_matrix.values,
    min_weight=0.05, max_weight=0.30,
    sector_mapping=sector_mapping,
    sector_limits=sector_limits
)
for asset, weight in zip(asset_names, sector_constrained['weights']):
    sector = sector_mapping[asset_names.index(asset)]
    print(f"  {asset} ({sector}): {weight:.4f} ({weight:.2%})")

# Check sector allocations
print("\nSector Allocation Check:")
for sector in set(sector_mapping.values()):
    indices = [i for i, s in sector_mapping.items() if s == sector]
    allocation = sum(sector_constrained['weights'][i] for i in indices)
    limit = sector_limits[sector]
    status = "✓" if allocation <= limit + 0.001 else "✗"
    print(f"  {sector}: {allocation:.2%} (limit: {limit:.0%}) {status}")

print(f"\nSharpe Ratio: {sector_constrained['sharpe_ratio']:.4f}")

# Comparison
print("\n4. Constraint Impact Summary:")
print("-" * 40)
print(f"{'Constraint':<30} {'Sharpe Ratio':>12}")
print(f"{'Unconstrained':<30} {unconstrained['sharpe_ratio']:>12.4f}")
print(f"{'Position Limits':<30} {pos_limited['sharpe_ratio']:>12.4f}")
print(f"{'Position + Sector':<30} {sector_constrained['sharpe_ratio']:>12.4f}")

---

## Question 9: Risk Parity Portfolio Construction

**Interview Question:** *"Explain the Risk Parity approach to portfolio construction. How does it differ from mean-variance optimization and how would you implement it?"*

### Key Concepts:

**Risk Parity** allocates based on equal risk contribution from each asset:

$$RC_i = w_i \cdot \frac{\partial \sigma_p}{\partial w_i} = w_i \cdot \frac{(\Sigma w)_i}{\sigma_p}$$

**Objective:** Each asset contributes equally to total portfolio risk:
$$RC_i = \frac{\sigma_p}{n} \quad \forall i$$

**Advantages over MVO:**
- Does not require expected return estimates (error-prone)
- More diversified by construction
- Historically robust out-of-sample

**Disadvantage:**
- May require leverage if assets have different volatilities

In [ ]:
# Question 9: Risk Parity Portfolio

def calculate_risk_contributions(weights, cov_matrix):
    """
    Calculate risk contribution of each asset.
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights
    cov_matrix : array-like
        Covariance matrix
        
    Returns:
    --------
    tuple : (marginal risk contributions, risk contributions, % risk contributions)
    """
    weights = np.array(weights)
    cov_matrix = np.array(cov_matrix)
    
    # Portfolio volatility
    port_vol = np.sqrt(weights @ cov_matrix @ weights)
    
    # Marginal risk contribution: ∂σ/∂w
    marginal_contrib = (cov_matrix @ weights) / port_vol
    
    # Risk contribution: w * (∂σ/∂w)
    risk_contrib = weights * marginal_contrib
    
    # Percentage risk contribution
    pct_risk_contrib = risk_contrib / port_vol
    
    return marginal_contrib, risk_contrib, pct_risk_contrib

def risk_parity_portfolio(cov_matrix, target_risk_contrib=None):
    """
    Find the Risk Parity Portfolio.
    
    Parameters:
    -----------
    cov_matrix : array-like
        Covariance matrix
    target_risk_contrib : array-like, optional
        Target risk contribution for each asset (default: equal)
        
    Returns:
    --------
    dict : Risk parity portfolio results
    """
    cov_matrix = np.array(cov_matrix)
    n = cov_matrix.shape[0]
    
    if target_risk_contrib is None:
        target_risk_contrib = np.ones(n) / n  # Equal risk contribution
    
    # Objective: minimize squared difference from target risk contributions
    def objective(w):
        port_vol = np.sqrt(w @ cov_matrix @ w)
        if port_vol < 1e-10:
            return 1e10
        
        marginal_contrib = (cov_matrix @ w) / port_vol
        risk_contrib = w * marginal_contrib
        pct_risk_contrib = risk_contrib / port_vol
        
        # Squared error from target
        return np.sum((pct_risk_contrib - target_risk_contrib) ** 2)
    
    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds (long only)
    bounds = tuple((0.001, 1) for _ in range(n))  # Small min to avoid zero weights
    
    # Initial guess: inverse volatility weights
    vols = np.sqrt(np.diag(cov_matrix))
    init_weights = (1 / vols) / np.sum(1 / vols)
    
    # Optimize
    result = minimize(
        objective,
        init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000, 'ftol': 1e-12}
    )
    
    weights = result.x
    port_vol = np.sqrt(weights @ cov_matrix @ weights)
    _, risk_contrib, pct_risk_contrib = calculate_risk_contributions(weights, cov_matrix)
    
    return {
        'weights': weights,
        'volatility': port_vol,
        'risk_contributions': risk_contrib,
        'pct_risk_contributions': pct_risk_contrib,
        'success': result.success
    }

print("Question 9: Risk Parity Portfolio")
print("=" * 50)

# Calculate risk parity portfolio
rp = risk_parity_portfolio(cov_matrix.values)

print("\n1. Risk Parity Portfolio Weights:")
print("-" * 40)
for asset, weight in zip(asset_names, rp['weights']):
    print(f"  {asset}: {weight:.4f} ({weight:.2%})")

print("\n2. Risk Contributions:")
print("-" * 40)
print(f"{'Asset':<12} {'Weight':>10} {'Risk Contrib':>15} {'% of Risk':>12}")
for i, asset in enumerate(asset_names):
    print(f"{asset:<12} {rp['weights'][i]:>10.4f} {rp['risk_contributions'][i]:>15.6f} {rp['pct_risk_contributions'][i]:>12.2%}")

print(f"\nPortfolio Volatility: {rp['volatility']:.4f} ({rp['volatility']:.2%})")
print(f"Sum of Risk Contributions: {sum(rp['risk_contributions']):.6f}")

# Compare with equal weight and minimum variance
print("\n3. Comparison of Portfolio Approaches:")
print("-" * 50)

# Equal weight risk contributions
eq_mrc, eq_rc, eq_prc = calculate_risk_contributions(equal_weights, cov_matrix.values)
eq_vol = calculate_portfolio_volatility(equal_weights, cov_matrix.values)

# Min variance risk contributions
mv_mrc, mv_rc, mv_prc = calculate_risk_contributions(
    numerical_long['weights'], cov_matrix.values
)

print(f"\n{'Portfolio':<20} {'Volatility':>12}")
print(f"{'Equal Weight':<20} {eq_vol:>12.4f}")
print(f"{'Risk Parity':<20} {rp['volatility']:>12.4f}")
print(f"{'Min Variance':<20} {numerical_long['volatility']:>12.4f}")

# Visualize risk contributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Equal weight
axes[0].bar(asset_names, eq_prc * 100)
axes[0].axhline(y=20, color='r', linestyle='--', label='Target (20%)')
axes[0].set_title('Equal Weight\nRisk Contributions')
axes[0].set_ylabel('% of Portfolio Risk')
axes[0].legend()

# Risk parity
axes[1].bar(asset_names, rp['pct_risk_contributions'] * 100)
axes[1].axhline(y=20, color='r', linestyle='--', label='Target (20%)')
axes[1].set_title('Risk Parity\nRisk Contributions')
axes[1].set_ylabel('% of Portfolio Risk')
axes[1].legend()

# Min variance
axes[2].bar(asset_names, mv_prc * 100)
axes[2].axhline(y=20, color='r', linestyle='--', label='Target (20%)')
axes[2].set_title('Min Variance\nRisk Contributions')
axes[2].set_ylabel('% of Portfolio Risk')
axes[2].legend()

for ax in axes:
    ax.set_ylim(0, 50)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---

## Question 10: Black-Litterman Model Basics

**Interview Question:** *"Explain the Black-Litterman model. What problems does it solve and how does it combine market equilibrium with investor views?"*

### Key Concepts:

The **Black-Litterman Model** (1992) addresses MVO's sensitivity to expected return inputs by:
1. Starting from **market equilibrium returns** (implied by market cap weights)
2. **Incorporating investor views** with confidence levels
3. Producing **stable, intuitive portfolios**

### Formula:

**Equilibrium Returns (CAPM):**
$$\Pi = \delta \Sigma w_{mkt}$$

Where:
- $\delta$ = risk aversion coefficient
- $\Sigma$ = covariance matrix  
- $w_{mkt}$ = market cap weights

**Posterior Returns (with views):**
$$E[R] = [(\tau\Sigma)^{-1} + P'\Omega^{-1}P]^{-1}[(\tau\Sigma)^{-1}\Pi + P'\Omega^{-1}Q]$$

Where:
- $P$ = view matrix (which assets the view is about)
- $Q$ = view returns
- $\Omega$ = uncertainty of views
- $\tau$ = scaling factor (typically 0.025-0.05)